In [1]:
import tensorflow as tf



# Tensors and Operations


In [4]:
tf.constant([[1.,2.,3.],[4.,5.,6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [6]:
t=tf.constant([[1.,2.,3.],[4.,5.,6.]])

In [7]:
##Similar to numpy,tf.Tensor has a shape and data type(dtype)
t.shape

TensorShape([2, 3])

In [8]:
t.dtype

tf.float32

In [9]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [11]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [2]:
##VAriables

v=tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [6]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [7]:
v[:,2].assign([0,1])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [ ]:
####Custom Loss functions

def huber_fn(y_true,y_pred):
  error=y_true-y_pred
  is_small_error=tf.abs(error)<1
  squared_loss=tf.square(error)/2
  linear_loss=tf.abs(error)
  return tf.where(is_small_error,squared_loss,linear_loss)

##model.compile(loss=huber_fn,optimizer="nadam")


#While loading we need to provide a dictionary mapping the custom name with the object
##Example-model=keras.models.load_model("my_model_with_a_custom_loss.h5",custom_objects={"huber_fn":huber_fn})


def create_huber(threshold=1.0):
  def huber_fn(y_true,y_pred):
    error=y_true-y_pred
    is_small_error=tf.abs(error)<threshold
    squared_loss=tf.square(error)/2
    linear_loss=tf.abs(error)
    return tf.where(is_small_error,squared_loss,linear_loss)
  return huber_fn

##model.compile(loss=create_huber(2.0),optimizer="nadam")

##Example-model=keras.models.load_model("my_model_with_a_custom_loss.h5",custom_objects={"huber_fn":create_huber(2.0) })



In [ ]:
##Custom Loss using subclass

class HuberLoss(keras.losses.Loss):
  def __init__(self,threshold=1.0,**kwargs):
    self.thershold=threshold
    super().__init__(**kwargs)

  def call(self,y_true,y_pred):
    error=y_true-y_pred
    is_small_error=tf.abs(error)<self.threshold
    squared_loss=tf.square(error)/2
    linear_loss=self.threshold*tf.abs(error)-self.threshold**2/2
    return tf.where(is_small_error,squared_loss,linear_loss)

  def get_config(self):
    base_config=super().get_config()
    return {**base_config,"threshold":self.thershold}     

In [ ]:
##Custom Activation Functions,Initializers,Regularizers and Constraints

def my_softplus(z):
  return tf.math.log(tf.exp(z)+1.0)

def glorot_initializer(shape,dtype=tf.float32):
  stddev=tf.sqrt(2./(shape[0]+shape[1]))


def my_l1_regularizer(weights):
  return tf.reduce_sum(tf.abs(0.01*weights))

def my_positive_weights(weights):
  return tf.where(weights<0,tf.zeros_like(weights),weights) 

##IMPLEMENTING THEM

layer=keras.layers.Dense(30,activation=my_softplus,
                         kernel_initializer=my_glorot_initializer,
                         kernel_regularizer=my_l1_regularizer,
                         kernel_constraint=my_positive_weights)









In [ ]:
##Custom Layers
##Custom layer without any weights

exponential_layer=keras.layers.Lambda(lambda x:tf.exp(x))


##Custom Layer with weights

class MyDense(keras.layers.Layer):

  def __init__(self,units,activation=None,**kwargs):
    super().__init__(**kwargs)
    self.units=units
    self.activation=keras.activations.get(activation)

  def build(self,batch_input_shape):
    self.kernel=self.add_weight(name="kernel",shape=[batch_input_shape[-1],self.units],initializer="glorot_normal")  
  
    self.bias=self.add_weight(name="bias",shape=[self.units],initializer="zeros")
    super().build(batch_input_shape)
  
  def call(self,X):
    return self.activation(X@self.kernel+self.bias)


  def compute_output_shape(self,batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1]+[self.units])

  def get_config(self):
    base_config=super().get_config()
    return {**base_config,"units":self.units,"activation":keras.activations.serialize(self.activation)}



In [ ]:

##Custom Model

class ResidualBlock(keras.layers.Layer):
  def __init__(self,n_layers,n_neurons,**kwargs):
    super().__init__(**kwargs)
    self.hidden=[keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="ne_normal") for _ in range(n_layers)]


  def call(self,inputs):
    z=inputs
    for layer in self.hidden:
      z=layer(z)
    return inputs+z


class ResidualRegressor(keras.models.Model):
  def __init__(self,output_dim,**kwargs):
    self.hidden1=keras.layers.Dense(30,activation="elu","kernel_initializer"="he_normal")

    self.block1=ResidualBlock(2,30)
    self.block2=ResidualBlock(2,30)
    self.out=keras.layers.Dense(output_dim)

  def call(self,inputs):
    z=self.hidden1(inputs)
    for _ in range(1+3):
      z=self.block1(z)
    z=self.block2(z)
    return self.   out(z)  

In [19]:
x=["252 - 121","22-2","121"]

x_gen=[d.split('-')[1].strip() if "-" in d else d  for d in x ]

In [20]:
x_gen

['121', '2', '121']

In [11]:
"123-123".split("-")

['123', '123']

In [12]:
"-" in "213-12"

True